In [5]:
# fast correlation based filter python implementation (FCBF)
# https://www.aaai.org/Papers/ICML/2003/ICML03-111.pdf

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from math import log, e
from scipy import stats
from collections import OrderedDict
from sklearn.metrics import mutual_info_score

# save load_iris() sklearn dataset to iris
iris = load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])


In [6]:
df.head()
# shows we have 4 features.. low dimensional but will be fine for now

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [13]:
# entropy calculation
def entropy2(x, base=2):

    if len(x) <= 1:
        return 0

    value, counts = np.unique(x, return_counts=True)
    probs = counts / len(x)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return(ent)

def entropy3(x, base = 2):
    value, count = np.unique(x, return_counts=True)
    return(stats.entropy(count, base=base))

In [14]:
# entropy functions all seem to output different things?
# also, is running a np.unique() the right thing to do?
# use 2 or 3
print(entropy2(df['sepal length (cm)']))
print(entropy3(df['sepal length (cm)']))

4.822018088381165
4.822018088381166


In [15]:
# information gain
def _joint_entropy(x, y):
    return(mutual_info_score(x, y))

def _conditional_entropy(x, y, base=2):
    # H(X|Y) = H(X,Y) - H(Y)
    # need to define the joint entropy
    
    # TODO: develop testing for size of arrays
    # TODO: develop testing for input types
    
    H_xy = _joint_entropy(x, y)
    H_y = entropy2(y)

    # TODO: calculate conditonal entropy here and check val.. are there bounds?
    # TODO: investigate potential optimisations for corner cases

    return(H_xy - H_y)


def information_gain(x, y):
    
    H_x = entropy2(x)
    H_xy = _conditional_entropy(x, y)
    
    # TODO: testing
    return(H_x - H_xy)

In [16]:
# calculate the symmetric uncertainty value
def symmetrical_uncertainty(x, y):
    return(2 * information_gain(x, y)/ entropy2(x) + entropy2(y))

In [18]:
# calculate the symmetrical uncertainty

features = df.iloc[:, 0:-1]
target = df.iloc[:, -1]

def calc_sprime(features, target):
    s_prime = {}
    for name, feature in features.iteritems():
        print("calculating for {} and target".format(name))
        tmp_su = symmetrical_uncertainty(feature.values, target.values)
        if(tmp_su >= 0.01): # random delta, should make to variable and change this later
            s_prime[name] = tmp_su
    s_prime = sorted(s_prime.items(), key = lambda x: x[1], reverse=True)    
    return(s_prime)

s_prime = calc_sprime(features, target)
print(s_prime)

calculating for sepal length (cm) and target
calculating for sepal width (cm) and target
calculating for petal length (cm) and target
calculating for petal width (cm) and target
[('sepal width (cm)', 4.198593345781159), ('sepal length (cm)', 3.9902350000624467), ('petal width (cm)', 3.8750376849519843), ('petal length (cm)', 3.816377686393965)]


In [ ]:
# get next element from the s_prime list
# need this in order to compare two subsequent features
# TODO: potential room for optimisation

# def get_next_element(cur_index, cur_list):
    # check if current element is the last element